---
        
💡 **NOTE**: We will want to use a GPU to run both Llama2 as well as BERTopic for this use case. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

We will start by installing a number of packages that we are going to use throughout this example:

In [ ]:
%%capture
!pip install transformers datasets accelerate bitsandbytes xformers langchain sentence_transformers autotrain-advanced faiss-gpu

# 🤗 HuggingFace Hub Credentials
Before we can load in Llama2 using a number of tricks, we will first need to accept the License for using Llama2. The steps are as follows:


* Create a HuggingFace account [here](https://huggingface.co)
* Apply for Llama 2 access [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)
* Get your HuggingFace token [here](https://huggingface.co/settings/tokens)

After doing so, we can login with our HuggingFace credentials so that this environment knows we have permission to download the Llama 2 model that we are interested in.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# 🦙 **Llama 2**

Now comes one of the more interesting components of this tutorial, how to load in a Llama 2 model on a T4-GPU!

We will be focusing on the `'meta-llama/Llama-2-13b-chat-hf'` variant. It is large enough to give interesting and useful results whilst small enough that it can be run on our environment.

We start by defining our model and identifying if our GPU is correctly selected. We expect the output of `device` to show a cuda device:

# 🗨️ **Retrieval Augmented Generation (RAG)**

In RAG, a knowledge base, like Wikipedia, is converted to numerical representations to capture its meaning, called embeddings. These embeddings are stored in a vector database so that the information can easily be retrieved.

<br>
<div>
<img src="https://cdn-images-1.medium.com/v2/resize:fit:1800/1*sRqMlpbsHhOAxsncsqlEHw.png" width="1250"/>
</div>

In [ ]:
# Our tiny knowledge base
knowledge_base = [
    "On July 18, 2023, in partnership with Microsoft, Meta announced LLaMA-2, the next generation of LLaMA." ,
    "Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ",
    "The fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases.",
    "Meta trained and released LLaMA-2 in three model sizes: 7, 13, and 70 billion parameters.",
    "The model architecture remains largely unchanged from that of LLaMA-1 models, but 40% more data was used to train the foundational models.",
    "The accompanying preprint also mentions a model with 34B parameters that might be released in the future upon satisfying safety targets."
]
with open(r'knowledge_base.txt', 'w') as fp:
    fp.write('\n'.join(knowledge_base))

### Embedding Model

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Embedding Model for converting text to numerical representations
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

### Create Database

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

# Load documents and split them
documents = TextLoader("knowledge_base.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Create local vector database
db = FAISS.from_documents(docs, embedding_model)

### RAG

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

# Load LLM into LangChain
llm = HuggingFacePipeline(pipeline=generator)

# RAG Pipeline
rag = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=db.as_retriever()
)

In [ ]:
print(llm('What is Llama 2?'))



Llama 2 is a cryptocurrency that was created as a parody of the popular cryptocurrency Dogecoin. It was designed to be a more serious and less meme-focused alternative to Dogecoin, with a focus on privacy and security.

Llama 2 uses a different mining algorithm than Dogecoin, called "Scrypt" instead of "Scrypt-N." This makes it more resistant to ASIC mining, which is a type of mining that uses specialized hardware to solve the mining algorithm quickly and efficiently. As a result, Llama 2 is more decentralized and less vulnerable to centralization, since it is harder for large mining operations to dominate the network.

Llama 2 also has a number of other features that set it apart from Dogecoin, including:

* Privacy-focused transactions: Llama 2 includes features like ring signatures and zero-knowledge proofs to help protect the privacy of users.
* Decentralized governance: Llama 2 has a decentralized governance system that allows holders of the cryptocurrency to vote on proposals f

In [ ]:
rag('What is Llama 2?')

{'query': 'What is Llama 2?',
 'result': ' Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) announced by Meta in partnership with Microsoft on July 18, 2023.'}

In [ ]:
llm('When was Llama 2 released?')

'\nLlama 2 was released on May 18, 2019.'

In [ ]:
rag('When was Llama 2 released?')

{'query': 'When was Llama 2 released?',
 'result': ' Llama 2 was released on July 18, 2023.'}